# 3年秋プロジェクト
## 自然言語処理

In [ ]:
# サイトから特定の文章を取得する
# ※※サーバー攻撃と処理時間の問題点からこれは実行しない※※


# みんなの大学情報

# MARCH大学群
univs_name = {
    20293: 'meiji',
    20192: 'aoyama',
    20298: 'rikkyo',
    20241: 'tyuo',
    20287: 'hosei'
}
# 各大学の学部リスト
univs_groups = {
    20293: [824, 825, 826, 827, 828, 829, 830, 831, 832, 833],
    20192: [469, 470, 471, 472, 473, 474, 475, 476, 477, 2279, 2563],
    20298: [854, 855, 856, 857, 858, 859, 860, 861, 862, 863],
    20241: [656, 657, 658, 659, 660, 661, 2565, 2566],
    20287: [794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 2328]
}
# 取り出したいクラス
Class = '.mod-reviewList-txt'


def univ_reviews(univ, group):
    '''レビューサイトをスクレイピングし特定のクラスを取得'''
    
    reviews = ''
    p = 1
    while True:
        url = 'https://www.minkou.jp/university/school/review/' + str(univ) + '/' + str(group) + '/page=' + str(p) + '/'
        html = requests.get(url)
        contents = BeautifulSoup(html.content, "html.parser")
        if len(contents.select(Class)) == 0:
            break
        for text in contents.select(Class):
            reviews += text.getText()
        p += 1
        time.sleep(1)
    return reviews

def univs_reviews(univs):
    '''各大学の各学部のレビューを取得し辞書型をファイルに保存'''
    
    for univ in univs:
        print(univs_name[univ], univ)
        univ_revs = {}
        for group in univs[univ]:
            print(group)
            # 1つの大学に対して、各学部のレビューを取得
            univ_revs[group] = univ_reviews(univ, group)
        # 取得したレビューを各大学ごとのpickleファイルに辞書型で保存
        with open(univs_name[univ]+".pkl", "wb") as f:
            pickle.dump(univ_revs, f)
    return



In [ ]:
# 初期準備
# import, リスト・辞書の定義

import requests
from bs4 import BeautifulSoup
from janome.tokenizer import Tokenizer
import math
import numpy as np
import time
from gensim import models
import pickle


# 以下手動で入力　自動取得法を模索 class="schMod-departmentList-name"でとってこれそう
univs_name = {
    20293: ['meiji', '明治大学'],
    20192: ['aoyama', '青山学院大学'],
    20298: ['rikkyo', '立教大学'],
    20241: ['tyuo', '中央大学'],
    20287: ['hosei', '法政大学']
}
undergraduates_name = {
    824: '法学部', 825: '政治経済学部', 826: '経営学部', 827: '商学部', 828: '文学部', 829: '情報コミュニケーション学部', 
    830: '理工学部', 831: '農学部', 832: '国際日本学部', 833: '総合数理学部',
    469: '法学部', 470: '経済学部', 471: '経営学部', 472: '文学部', 473: '国際政治経済学部', 474: '教育人間科学部', 
    475: '総合文化政策学部', 476: '理工学部', 477: '社会情報学部', 2279: '地球社会共生学部', 2563: 'コミュニティ人間科学部',
    854: '法学部', 855: '経済学部', 856: '文学部', 857: '理学部', 858: '社会学部', 859: '経営学部', 
    860: '異文化コミュニケーション学部', 861: '観光学部', 862: 'コミュニティ福祉学部', 863: '現代心理学部',
    656: '法学部', 657: '経済学部', 658: '商学部', 659: '文学部', 660: '総合政策学部', 661: '理工学部',
    2565: '国際経営学部', 2566: '国際情報学部',
    794: '法学部', 795: '経営学部', 796: '文学部', 797: '国際文化学部', 798: '人間環境学部', 799: 'キャリアデザイン学部',
    800: 'デザイン工学部', 801: '情報科学部', 802: '理工学部', 803: '生命科学部', 804: 'スポーツ健康学部', 
    805: '経済学部', 806: '現代福祉学部', 807: '社会学部', 2328: 'グローバル教養学部',
}
univs_groups = {
    20293: [824, 825, 826, 827, 828, 829, 830, 831, 832, 833],
    20192: [469, 470, 471, 472, 473, 474, 475, 476, 477, 2279, 2563],
    20298: [854, 855, 856, 857, 858, 859, 860, 861, 862, 863],
    20241: [656, 657, 658, 659, 660, 661, 2565, 2566],
    20287: [794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 2328]
}



In [ ]:
# ファイルの読み込み

# ローカルに保存した法政のレビュー解析結果(pklファイル)を読み込む
with open('meiji.pkl', 'rb') as f:
    meiji_revs = pickle.load(f)

with open('aoyama.pkl', 'rb') as f:
    aoyama_revs = pickle.load(f)

with open('rikkyo.pkl', 'rb') as f:
    rikkyo_revs = pickle.load(f)

with open('tyuo.pkl', 'rb') as f:
    tyuo_revs = pickle.load(f)

with open('hosei.pkl', 'rb') as f:
    hosei_revs = pickle.load(f)
    
univs_revLsit = [meiji_revs, aoyama_revs, rikkyo_revs, tyuo_revs, hosei_revs]



# 各大学、各学部のレビューにおける形態素解析済みの単語リストと単語出現回数リスト
# pklに保存しているものをopenする
with open('revw.pkl', 'rb') as f:
    revw = pickle.load(f)

with open('revwc.pkl', 'rb') as f:
    revwc = pickle.load(f)

# レビューの分かち書きリスト
with open('wakati_docs.pkl', 'rb') as f:
    wakati_docs = pickle.load(f)

# 大学学部のレビューの学習モデル
rev_model = models.Word2Vec.load('univ_review.model')

# 白ヤギモデル・大学学部レビューモデルの融合学習モデルの読み込み
fus_model = models.Word2Vec.load('gensim_univs.model')

# 大学学部と各レビューのdoc2vecモデル
revs_model = models.Doc2Vec.load('univs_reviews.model')

# 学習させる文書をレビュー全文の分かち書きとしたdoc2vecモデル
wakati_model = models.Doc2Vec.load('wakati_reviews.model')



In [ ]:
# 関数等定義

def tf(words):
    tf_words = {}
    s = sum(words.values())
    for key, value in words.items():
        tf_words[key] = value / s
    return tf_words

def df(messages):
    df_words = {}
    for prof, mesg in messages.items():
        for key in mesg.keys():
            num = df_words.get(key)
            if num == None:
                df_words[key] = 1
            else:
                df_words[key] = num + 1
    return df_words

def tfidf(messages):
    df_words = df(messages)
    tf_tfidf = {}
    size = len(messages)
    for prof, mesg in messages.items():
        tf_scores = tf(mesg)
        tfidf_scores = {}
        for key, value in tf_scores.items():
            tfidf_scores[key] = value * math.log2(size / df_words[key])
        tf_tfidf[prof] = (tf_scores, tfidf_scores)
    return tf_tfidf



# 大学学部名のリスト化(最後尾はテスト用)
# 各レビュー文の分かち書きリスト(文書作成) tfidfの値が0の単語をストップワードにして、それ以外をリストに追加する
# 結果から、「青山学院大学コミュニティ人間科学部」だけ38個であり、データ数が他と比べて少なすぎる
# 青山学院大学コミュニティ人間科学部はレビューが少なすぎるため今回は除外
univsName = []
docs = []
stop_words = []
for i, u in enumerate(univs_groups):
    univ = univs_name[u][1]
    for g in univs_groups[u]:
        if g == 2563: continue
        undergraduate = undergraduates_name[g]
        name = univ + ' ' + undergraduate
        univsName.append(name)

        doc = []
        stop_word = []
        tf_idf = tfidf(revwc[i])[g][1]
        tf_idf_keys = list(tf_idf.keys())
        tf_idf_values = list(tf_idf.values())
        for k in range(len(tf_idf)):
            if tf_idf_values[k] == 0.0:
                stop_word.append(tf_idf_keys[k])
                stop_words.append(stop_word)
        doc = [d for d in revw[i][g] if d not in stop_word]
        docs.append(doc)


# 各学部のレビュー全文を分かち書きしてリスト化
# wakati_docs = []
# tw = Tokenizer(wakati=True)
# for i, univ in enumerate(univs_groups):
#     for g in univs_groups[univ]:
#         if g == 2563: continue
#         token = tw.tokenize(univs_revLsit[i][g])
#         wakati_doc = list(token)
#         wakati_docs.append(wakati_doc)


# 入力文を分かち書きする
def wakati_sentence(text):
    tw = Tokenizer(wakati=True)
    token = tw.tokenize(text)
    tokenList = list(token)
    return tokenList


# 文章を形態素解析(分かち書き)する　単語群と出現回数リストを返す関数
def wakati_count_words(contents):
    t1 = Tokenizer()
    words = []
    words_count = {}
    for token in t1.tokenize(contents):
        parts = token.part_of_speech.split(',')
        if('名詞' in parts
            or '動詞' in parts
            or '形容詞' in parts
            or '形容動詞' in parts):
            base = token.base_form
            words.append(base)
            rec = words_count.get(base)
            if rec == None:
                words_count[base] = 1
            else:
                words_count[base] = rec + 1
    return words, words_count


# 白ヤギ+レビューのword2vecモデルを用いて、単語の似ているワードをリスト化
def similar_words(text, model, n):
    sentence = wakati_sentence(text)
    words, words_count = wakati_count_words(text)
    wordsList = sentence
    for word in words:
        sim_words = model.wv.most_similar(word, topn=n)
        for w in range(len(sim_words)):
            wordsList.append(sim_words[w][0])
    return wordsList



# レビューに大学学部名を付与するためのクラス
class LabeledListSentence(object):
    def __init__(self, words_list, labels):
        self.words_list = words_list
        self.labels = labels
    def __iter__(self):
        for i, words in enumerate(self.words_list):
            yield models.doc2vec.LabeledSentence(words, ['%s' % self.labels[i]])


# docsに入力となるテキストを新たに追加して、それを込みで学習させる
def testData_train(data, docs, univs):
    univs.append('test')
    docs.append(data)
    sentences = LabeledListSentence(docs, univs)
    model = models.Doc2Vec(dm=1, size=200, min_count=5, alpha=0.025, iter=20, workers=4)
    model.build_vocab(sentences)
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    univs = model.docvecs.offset2doctag
    univs.pop()
    docs.pop()
    return model



# 入力文から、最も近い・合っている大学学部を出力(提案)する
def output_similar_univ(input_text, model, n):
    text = similar_words(input_text, fus_model, 10)

    # 新規文書のモデルにおけるベクトルを作成する
    input_vec = model.infer_vector(text)
    
    # 新規文書のベクトルと近いものを出力 ⇒ 条件や要件に近い・合っている大学学部を提案する
    output_univs = model.docvecs.most_similar([input_vec], topn=n)

    return output_univs

#### 実験

label:test, doc:入力文　を追加したsentencesを再学習して近いベクトルを比較する

In [ ]:
input_text = 'プログラミングを学べる、CG、音声処理、画像処理、情報、パソコンに詳しくなりたい、設備がきれい'
text = similar_words(input_text, fus_model, 10)
test_model = testData_train(text, docs, univsName)

In [ ]:
# 54の文書が入力したテキスト文
# 理論上は、こちらが入力した条件(text)に最も近い(適切な)大学学部を出力する
# 精度は不明

test_model.docvecs.most_similar(53, topn=10)

In [ ]:
test_model.docvecs.most_similar(0, topn=10)

2022.1.10　精度が低い
考えられる原因や改善点
・Doc2Vecの文書比較の中身や計算方法を理解しきれていないため最適ではない可能性
・Doc2Vecのパラメータの変更やエポック数分の繰り返し学習の実行
・各大学学部の文書となるdocsの中身を見たところ、ゴミが多い(思う、する、ある、数字や記号...)
　→その単語単体では意味を持たないような、もしくはその学部を特徴づける語ではないものが多いため、精度が下がっているのでは
　→現在のdocsの作成方法はレビューを名詞・動詞・形容詞・形容動詞で分かち書きしているだけ
　→出現回数やtf-idfでふるいにかけることで精度は向上するかもしれない、ストップワードの活用など

Doc2Vecのライブラリを調べたりして、最適な手法を模索する

infer_vectorで未知の入力文(今回のtext)をモデルにおけるベクトルで表す
得られたベクトルに近いものを出力する

多分、上記のtestData_trainでやるよりも、適切な手法だと思われる。
testdata分の文書やベクトルが他の学部の結果に影響してしまうから。

ただ、やはり精度は十分とは言えず、docsの修正が必須だと考えられる

1/11 0:10 docsを修正　tfidfの処理で特徴語でないものを除いた分かち書きリスト(ゴミを削除)


今後の改善点としては
doc2vecモデルの学習精度を上げるためにパラメータ等を工夫する
docsはゴミを消したので、基本的にこれ以上は無理と仮定

In [ ]:
# 理科大　理工学部
test_doc = univ_reviews(20267, 729)

各学部のmodelを作成して、infer_vectorによるベクトル生成で比較する方法

docs は名詞動詞形容詞形容動詞を取得しtf-idfで特徴のない語を除去したリスト

In [ ]:
# 大学学部とレビューのDoc2Vecのモデル構築と学習

sentences = []
for i in range(len(univsName)):
    doc = docs[i]
    tag = [univsName[i]]
    sentences.append(models.doc2vec.TaggedDocument(doc, tag))

revs_model = models.Doc2Vec(sentences, dm=1, size=200, min_count=5, alpha=0.025, iter=20, workers=4)

# sentences = LabeledListSentence(docs, univsName)
# revs_model = models.Doc2Vec(size=300, dm=1, window=10, alpha=0.025, min_count=5, iter=20, workers=4)
# revs_model.build_vocab(sentences)
# revs_model.train(sentences, total_examples=revs_model.corpus_count, epochs=revs_model.iter)

revs_model.save('univs_reviews.model')

In [ ]:
# 大学学部と各レビューのdoc2vecモデル
revs_model = models.Doc2Vec.load('univs_reviews.model')

In [ ]:
input_text = 'プログラミングを学べる、CG、音声処理、画像処理、情報、パソコンに詳しくなりたい、設備がきれい'
output_similar_univ(input_text, revs_model, 10)

In [ ]:
print('「課題」と似ている単語')
print(revs_model.most_similar('課題'))
print()
print('「プログラミング」と似ている単語')
print(revs_model.most_similar('プログラミング'))

In [ ]:
print('「明治大学法学部」と似ている大学学部')
print(revs_model.docvecs.most_similar(0))
print()
print('「法政大学情報科学部」と似ている大学学部')
print(revs_model.docvecs.most_similar(45))

In [ ]:
text = wakati_sentence(test_doc)

# 新規文書のモデルにおけるベクトルを作成する
input_vec = revs_model.infer_vector(text)

# 新規文書のベクトルと近いものを出力 ⇒ 条件や要件に近い・合っている大学学部を提案する
revs_model.docvecs.most_similar([input_vec], topn=10)

docs をレビュー全文の分かち書きにして実験

In [ ]:
sentences = []
for i in range(len(univsName)):
    doc = wakati_docs[i]
    tag = [univsName[i]]
    sentences.append(models.doc2vec.TaggedDocument(doc, tag))

wakati_model = models.Doc2Vec(sentences, dm=1, size=200, min_count=5, alpha=0.025, iter=20, workers=4)

wakati_model.save('wakati_reviews.model')

In [ ]:
wakati_model = models.Doc2Vec.load('wakati_reviews.model')

In [ ]:
print('「課題」と似ている単語')
print(wakati_model.most_similar('課題'))
print()
print('「プログラミング」と似ている単語')
print(wakati_model.most_similar('プログラミング'))
print()
print()
print('「明治大学法学部」と似ている大学学部')
print(wakati_model.docvecs.most_similar(0))
print()
print('「法政大学情報科学部」と似ている大学学部')
print(wakati_model.docvecs.most_similar(45))

In [ ]:
input_text = 'プログラミングを学べる大学。CGとか音声処理とか画像処理の分野を勉強したい。IT系。パソコンに詳しくなりたい。'

print('「', input_text, '」 という条件に近い大学学部')
print()
output_similar_univ(input_text, wakati_model, 5)

In [ ]:
input_text = '偏差値そこそこ。広い、綺麗。おしゃれ。課題が少ない。教授が有名。友達が作りやすい。男女比のバランスが良い。ネームバリューが良い。就職に有利、就職先が大手。対面授業。立地が良い。'

print('「', input_text, '」 という条件に近い大学学部')
print()
output_similar_univ(input_text, wakati_model, 5)

In [ ]:
text = wakati_sentence(test_doc)

# 新規文書のモデルにおけるベクトルを作成する
input_vec = wakati_model.infer_vector(text)

# 新規文書のベクトルと近いものを出力 ⇒ 条件や要件に近い・合っている大学学部を提案する
wakati_model.docvecs.most_similar([input_vec], topn=10)

## 以下各手法まとめ

### tf-idf

In [ ]:
def wakati_count_words(contents):
    t1 = Tokenizer()
    words = []
    words_count = {}
    for token in t1.tokenize(contents):
        parts = token.part_of_speech.split(',')
        if('名詞' in parts
            or '動詞' in parts
            or '形容詞' in parts
            or '形容動詞' in parts):
            base = token.base_form
            words.append(base)
            rec = words_count.get(base)
            if rec == None:
                words_count[base] = 1
            else:
                words_count[base] = rec + 1
    return words, words_count

def univ_rev_lists(univs, uList):
    univs_words = []
    univs_words_count = []
    for i, univ in enumerate(univs):
        words = {}
        words_count = {}
        for group in univs[univ]:
            words[group], words_count[group] = wakati_count_words(uList[i][group])
        univs_words.append(words)
        univs_words_count.append(words_count)
    return univs_words, univs_words_count


def sort_values(words):
    return sorted(words.items(), key = lambda x: x[1], reverse = True)

def tf(words):
    tf_words = {}
    s = sum(words.values())
    for key, value in words.items():
        tf_words[key] = value / s
    return tf_words

def df(messages):
    df_words = {}
    for prof, mesg in messages.items():
        for key in mesg.keys():
            num = df_words.get(key)
            if num == None:
                df_words[key] = 1
            else:
                df_words[key] = num + 1
    return df_words

def tfidf(messages):
    df_words = df(messages)
    tf_tfidf = {}
    size = len(messages)
    for prof, mesg in messages.items():
        tf_scores = tf(mesg)
        tfidf_scores = {}
        for key, value in tf_scores.items():
            tfidf_scores[key] = value * math.log2(size / df_words[key])
        tf_tfidf[prof] = (tf_scores, tfidf_scores)
    return tf_tfidf

In [ ]:
# 各大学、各学部のレビューにおける形態素解析済みの単語リストと単語出現回数リスト
# revw[0]はmeijiのデータ　…
revw, revwc = univ_rev_lists(univs_groups, univs_revLsit)

with open("revw.pkl", "wb") as f:
    pickle.dump(revw, f)

with open("revwc.pkl", "wb") as f:
    pickle.dump(revwc, f)

In [ ]:
# if-idfで特徴語抽出　学部名が含まれる語は除く

all_revwc = [tfidf(revwc[i]) for i in range(len(univs_groups))]
for i, univ in enumerate(univs_groups):
    print(univs_name[univ][1])
    for group in univs_groups[univ]:
        words = list(all_revwc[i][group][1].keys())
        for n in range(len(words)):
            if words[n] in undergraduates_name[group]:
                all_revwc[i][group][1][words[n]] = 0
        print(undergraduates_name[group])
        print(sort_values(all_revwc[i][group][1])[:5])
        print()

### 分散表現

In [ ]:
# 単語にIDを付与
def create_corpus(words, corpus = [], word_to_id = {}, id_to_word = {}):
    corpus = []
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id: # 初めての単語なら
            new_id = len(word_to_id)  # ID を決めて
            word_to_id[word] = new_id # word_to_id に登録
            id_to_word[new_id] = word # id_to_word に登録

    corpus += [word_to_id[w] for w in words] # 文章をIDリストで表現

    return corpus, word_to_id, id_to_word


# 共起行列の計算
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix


# コサイン類似度
def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)


# 似ている単語の検索
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    if query not in word_to_id:
        print('%s is not found' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    vocab_size = len(id_to_word)

    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [ ]:
query = '楽しい'

for i, univ in enumerate(univs_groups):
    print(univs_name[univ][1])
    for group in univs_groups[univ]:
        corpus, word_to_id, id_to_word = create_corpus(revw[i][group])
        word_matrix = create_co_matrix(corpus, len(word_to_id))
        print(undergraduates_name[group])
        most_similar(query, word_to_id, id_to_word, word_matrix, 3)
        print()

### word2vec

In [ ]:
# 白ヤギさんが作った日本語word2vecモデル
model_path = 'latest-ja-word2vec-gensim-model/word2vec.gensim.model'
model = models.Word2Vec.load(model_path)

In [ ]:
# 各大学学部のレビュー文を一つの文字列にまとめる
reviews_txt = ''
for i, univ in enumerate(univs_groups):
    for group in univs_groups[univ]:
        reviews_txt += univs_revLsit[i][group]

# janomeで分かち書き
tw = Tokenizer(wakati=True)
token = tw.tokenize(reviews_txt)
reviews_wakati = ' '.join(list(token))

# 分かち書きされたテキストをファイルに書き込む
with open('rev_wakati.txt', 'w', encoding='utf-8_sig') as f:
    f.write(reviews_wakati)

# レビュー文のword2vecのモデル作成
docs = models.word2vec.LineSentence('rev_wakati.txt')
rev_model = models.Word2Vec(docs, size=100, min_count=5, window=5, iter=3)
rev_model.save('univ_review.model')


# 白ヤギさんのモデルに各大学学部の分かち書きレビューモデルを追加学習
docs = models.word2vec.LineSentence('rev_wakati.txt')
model.train(docs, total_examples=model.corpus_count, epochs=model.iter)
model.save('gensim_univs.model')

In [ ]:
# 大学学部のレビューの学習モデル
rev_model = models.Word2Vec.load('univ_review.model')

# 白ヤギ・学部レビューの融合学習モデル
modelV2 = models.Word2Vec.load('gensim_univs.model')

In [ ]:
text = '課題が少ない大学'
w,wc = wakati_count_words(text)

for x in w:
    print(modelV2.wv.most_similar(x))
    print()
    print(rev_model.wv.most_similar(x))
    print()

### Doc2Vec

In [ ]:
# 大学学部名のリスト化(最後尾はテスト用)、各レビュー文の分かち書きリスト(文書作成)

univsName = []
docs = []
for i, u in enumerate(univs_groups):
    univ = univs_name[u][1]
    for g in univs_groups[u]:
        undergraduate = undergraduates_name[g]
        name = univ + ' ' + undergraduate
        univsName.append(name)
        docs.append(revw[i][g])
univsName.append('testData')


In [ ]:
# 参考記事： http://qiita.com/okappy/items/32a7ba7eddf8203c9fa1

# レビューに大学学部を付与
class LabeledListSentence(object):
    def __init__(self, words_list, labels):
        self.words_list = words_list
        self.labels = labels

    def __iter__(self):
        for i, words in enumerate(self.words_list):
            yield models.doc2vec.LabeledSentence(words, ['%s' % self.labels[i]])


sentences = LabeledListSentence(docs, univsName)

revs_model = models.Doc2Vec(alpha=0.025, min_count=5, size=100, iter=20, workers=4)

revs_model.build_vocab(sentences)

revs_model.train(sentences, total_examples=revs_model.corpus_count, epochs=revs_model.iter)

revs_model.save('univs_reviews.model')

In [ ]:
revs_model = models.Doc2Vec.load('univs_reviews.model')
univsName = revs_model.docvecs.offset2doctag

In [ ]:

# レビューに大学学部名を付与するためのクラス
class LabeledListSentence(object):
    def __init__(self, words_list, labels):
        self.words_list = words_list
        self.labels = labels

    def __iter__(self):
        for i, words in enumerate(self.words_list):
            yield models.doc2vec.LabeledSentence(words, ['%s' % self.labels[i]])


# 白ヤギ+レビューのword2vecモデルを用いて、単語の似ているワードをリスト化
def similar_words(text, model, n):
    words, words_count = wakati_count_words(text)
    wordsList = []
    for word in words:
        sim_words = model.wv.most_similar(word, topn=n)
        for w in range(len(sim_words)):
            word = sim_words[w][0]
            wordsList.append(word)
    return wordsList



# docsに入力となるテキストを新たに追加して、それを込みで学習させる
def testData_train(data, docs, univs):
    docs.append(data)
    sentences = LabeledListSentence(docs, univs)
    model = models.Doc2Vec(alpha=0.025, min_count=5, size=100, iter=20, workers=4)
    model.build_vocab(sentences)
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    univs = model.docvecs.offset2doctag
    docs.pop()
    return model

In [ ]:
# 大学学部とレビューのDoc2Vecのモデル構築と学習 
# 繰り返し学習させる

sentences = LabeledListSentence(docs, univsName)

revs_model2 = models.Doc2Vec(sentences, dm=1, size=300, window=5, alpha=0.025, min_alpha=0.025,
                            min_count=5, sample=1e-6)

print('\n訓練開始')
for epoch in range(1000):
    #print('Epoch: ' , epoch+1)
    revs_model2.train(sentences, total_examples=revs_model2.corpus_count, epochs=revs_model2.iter)
    revs_model2.alpha -= (0.025 - 0.0001) / 999
    revs_model2.min_alpha = revs_model2.alpha
print('\n訓練終了')

revs_model2.save('univs_reviews2.model')
# 大学学部と各レビューのdoc2vecモデル
revs_model2 = models.Doc2Vec.load('univs_reviews2.model')
univsName = revs_model2.docvecs.offset2doctag
input_text = 'プログラミング、画僧処理や映像処理、パソコンを勉強したい'
text = similar_words(input_text, model, 20)

# 新規文書のモデルにおけるベクトルを作成する
input_vec = revs_model2.infer_vector(text)

# 新規文書のベクトルと近いものを出力
revs_model2.docvecs.most_similar([input_vec], topn=5)

#### 実験

In [ ]:
text = '課題が少ない大学'
test = similar_words(text)
test_model = testData_train(test, docs, univsName)

In [ ]:
w, wc = wakati_count_words(text)
for t in w:
    print(test_model.most_similar(t, topn=5))

In [ ]:
# 54の文書が入力したテキスト文
# 理論上は、こちらが入力した条件(text)に最も近い(適切な)大学学部を出力する
# 精度は不明

test_model.docvecs.most_similar(54, topn=5)

### トピックモデル LDA

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

In [ ]:
common_dictionary = Dictionary(docs)

In [ ]:
common_corpus = [common_dictionary.doc2bow(text) for text in docs]

In [ ]:
common_corpus

In [ ]:
lda = LdaModel(common_corpus, num_topics=5, id2word=common_dictionary.id2token, iterations=100000, gamma_threshold=0.000001)

In [ ]:
lda.print_topics()

In [ ]:
str = '法政大学の市ヶ谷キャンパスには今年度から使用されている大内山校舎という新しい校舎があります。また、3年ほど前にできた富士見ゲートという校舎もあり、どちらも多くの授業が開講されているので、新しい校舎の綺麗な環境で勉強することができます。特にトイレが明るくとても綺麗で、混み合ってしまうという難点はありますが私はとても気に入っています。現在古い校舎は工事中なので移動が難しいですが、そちらが終わればとても使いやすい綺麗なキャンパスになると思います。'
w,wc = wakati_count_words(str)

In [ ]:
vec = common_dictionary.doc2bow(w)

In [ ]:
print(lda[vec])

In [ ]:
lda.print_topic(10)